# Рекомендация тарифов мобильной связи

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Предоставлены данные о поведении клиентов, которые уже перешли на эти тарифы.

Необходимо построить модель с максимально большим значением accuracy (доля правильных ответов по крайней мере до 0.75).

* **сalls** — количество звонков,
* **minutes** — суммарная длительность звонков в минутах,
* **messages** — количество sms-сообщений,
* **mb_used** — израсходованный интернет-трафик в Мб,
* **is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откроем и изучим файл

In [1]:
# Подключаем библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
from tqdm import tqdm
from sklearn.dummy import DummyClassifier

In [ ]:
# Открываем файл
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [4]:
# Ознакомимся с данными
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В столбце masseges находится информация o количестве сообщейний, эти данные должны быть типа int.

In [6]:
# Изменим тип данных для messeges на int
df['messages'] = df['messages'].astype('int')

In [7]:
# Проверим наличие дубликатов
df.duplicated().sum()

0

In [8]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**ВЫВОДЫ:**

В таблице приведены данные о поведении клиентов оператора мобильной связи.

Данные выглядят корректно, пропусков и дубликатов нет. Тип данных в столбце messages заменили на int.

## Разобьем данные на выборки

In [10]:
# Разобьем данные в соотношении 60/20/20
train_df, lb_df = train_test_split(df, test_size=0.40, random_state=12345)
valid_df, test_df = train_test_split(lb_df, test_size=0.50, random_state=12345)

print('Размер тренериующей выборки', train_df.shape[0])
print('Размер валидационной выборки', valid_df.shape[0])
print('Размер тестовой выборки', test_df.shape[0])

Размер тренериующей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Предоставлены данные о поведении клиентов, которые уже перешли на эти тарифы.
Необходимо построить модель с максимально большим значением accuracy (доля правильных ответов по крайней мере до 0.75).

Так как конечной целью исследования является создание системы, которая правильно предлагает тариф клиенту на основании его поведения, то целевым признаком будет столбец is_ultra.

In [11]:
# извлечем признаки и целовые признаки для каждого датафрейма
train_df_features = train_df.drop(['is_ultra'], axis=1)
train_df_target = train_df['is_ultra']
valid_df_features = valid_df.drop(['is_ultra'], axis=1)
valid_df_target = valid_df['is_ultra']
test_df_features = test_df.drop(['is_ultra'], axis=1)
test_df_target = test_df['is_ultra']

**ВЫВОДЫ:**

Исходные данные были разбиты на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.

## Исследуем модели

Мы решаем задачу классификации, целевой признак является категориальным. Мы рассмотрим три модели: решающее дерево, случайный лес, логистическую регрессию.

### Decision Tree

In [12]:
best_dt_model = None
best_accuracy_dt_model = 0
best_depth_dt_model = 0

for depth in range(2, 25):
    dt_model = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    dt_model.fit(train_df_features, train_df_target)
    prediction_valid_dt_model = dt_model.predict(valid_df_features)
    accuracy_dt_model = accuracy_score(prediction_valid_dt_model, valid_df_target)
    if best_accuracy_dt_model < accuracy_dt_model:
        best_dt_model= dt_model
        best_depth_dt_model = depth
        best_accuracy_dt_model = accuracy_dt_model

In [13]:
f'Accuracy наилучшей модели на валидационной выборке: {best_accuracy_dt_model}'

'Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692'

In [14]:
'Глубина',best_depth_dt_model

('Глубина', 3)

### Random Forest


In [15]:
%%time

best_rf_model = None
best_accuracy_rf_model = 0
best_est_rf_model = 0
best_depth_rf_model = 0

for est in tqdm(range(10, 51, 10)):
    for depth in range(2,25):
        rf_model = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth)
        rf_model.fit(train_df_features, train_df_target)
        prediction_valid_rf_model = rf_model.predict(valid_df_features)
        accuracy_rf_model = accuracy_score(prediction_valid_rf_model, valid_df_target)
        if best_accuracy_rf_model < accuracy_rf_model:
            best_rf_model= rf_model
            best_depth_rf_model = depth
            best_est_rf_model = est
            best_accuracy_rf_model = accuracy_rf_model



100%|██████████| 5/5 [00:17<00:00,  3.60s/it]

CPU times: user 17.4 s, sys: 74.4 ms, total: 17.4 s
Wall time: 18 s


In [16]:
'Accuracy наилучшей модели на валидационной выборке:', best_accuracy_rf_model,

('Accuracy наилучшей модели на валидационной выборке:', 0.8087091757387247)

In [17]:
'Количество деревьев', best_est_rf_model,'Глубина',best_depth_rf_model

('Количество деревьев', 40, 'Глубина', 8)

### Logistic Regression


In [18]:
lr_model = LogisticRegression(random_state = 12345, max_iter= 1000, solver = 'lbfgs' )
lr_model.fit(train_df_features, train_df_target)
prediction_valid_lr_model = lr_model.predict(valid_df_features)
accuracy_lr_model = accuracy_score(prediction_valid_lr_model, valid_df_target)

In [19]:
'Accuracy модели на валидационной выборке:', accuracy_lr_model

('Accuracy модели на валидационной выборке:', 0.7107309486780715)

**ВЫВОДЫ:**

* Были инициализированы модели дерева решений, случайного леса и логистической регрессии.
* Были постореены модели с наилучшим значением аccuracy.
* Чем больше аccuracy, тем лучше. Наилучший результат дает модель случайного леса (0.81).

Надо учитывать, что несмотря на высорое качество работы, скорость работы может оказаться низкой. В таком случае нужно будет использоватьлогистическую регрессию с сопоставимым качеством работы, но высокой скоростью работы.


## Проверим модель на тестовой выборке

Лучшее значение параметра аccuracy было у модели случайного леса.

In [20]:
test_predictions = best_rf_model.predict(test_df_features)
accuracy = accuracy_score(test_df_target, test_predictions)
'Точность модели решающего леса на тестовой выборке',accuracy

('Точность модели решающего леса на тестовой выборке', 0.7962674961119751)

**ВЫВОДЫ:**

На стадии проверки подтвердилось, что самые качественные предстазания получаются с использованием модели решающего леса. А вот качество предсказаний логистической регрессии оказалось очень низким, меньше 0.75.

## Проверим модели на адекватность

Хорошую модель от плохой отличают с помощью метрик качества: accuracy(качество), precision(точность) и recall(полнота). Используется концепция для описания этих метрик в терминах ошибок классификации — confusion matrix (матрица ошибок).


In [21]:
test_predictions = best_rf_model.predict(test_df_features)

accuracy = accuracy_score(test_df_target, test_predictions)
precision = precision_score(test_df_target, test_predictions)
recall = recall_score(test_df_target, test_predictions)

print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall)

Accuracy = 0.7962674961119751 Precision = 0.75 Recall = 0.5320197044334976


Precision можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными. В нашем случае Precision = 0.75

Recall показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм. Recall = 0.53.

Precision и recall не зависят, в отличие от accuracy, от соотношения классов и потому применимы в условиях несбалансированных выборок. Обычно при оптимизации гиперпараметров алгоритма используется одна метрика, улучшение которой мы и ожидаем увидеть на тестовой выборке. Существует способ объединить precision и recall в агрегированный критерий качества. F-мера (в — среднее гармоническое precision и recall.




In [22]:
f_score = f1_score(test_df_target, test_predictions)
print('F-мера =', f_score)

F-мера = 0.622478386167147


**ВЫВОДЫ**
Хорошую модель от плохой отличают с помощью метрик качества: accuracy(качество), precision(точность) и recall(полнота). Для нашей модели случайного леса получились следующие значения:

Accuracy = 0.80
Precision = 0.75
Recall = 0.53

Еще одним параметром характеризующим адекватность является среднее гармоническое precision и recall - F-мера (0.62).

In [23]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_df_features, train_df_target)
dummy_clf.predict(train_df_features)
dummy_clf.score(train_df_features, train_df_target)

0.6924273858921162

## Итоговые выводы

Было проведено исследование для оператора мобильной связи «Мегалайн». Чтобы оптимально переводить пользователей на новые рарифы нужно было построить модель с максимально большим значением accuracy.

Были вывполнены следующие шаги:
* Файл с данными был открыт и изучен. Данные оказались корректными, пропусков и дубликатов не содержали.
* Исходные данные были разделены на обучающую, валидационную и тестовую выборки.
* Меняя гиперпараметры было изучено качество разных моделей.
* Качество моделей было проверено на тестовой выборке. Наилучший результат дает модель случайного леса (0.81).
* На стадии проверки подтвердилось, что самые качественные предстазания получаются с использованием модели решающего леса. А вот качество предсказаний логистической регрессии оказалось очень низким, меньше 0.75.
* Качество модели было проверено на вменяемость. с помощью метрик качества: accuracy(качество), precision(точность) и recall(полнота). Для нашей модели случайного леса получились следующие значения: Accuracy = 0.80 Precision = 0.75 Recall = 0.53. Еще одним параметром характеризующим адекватность является среднее гармоническое precision и recall - F-мера (0.62).
